In [1]:
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

import joblib

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv', skipinitialspace=True)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [10]:
x_cols = [c for c in df.columns if c != 'income']

X=df[x_cols]
y=df['income']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1234)

In [11]:
# mode: the highest frequency value
trn_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(trn_mode)

encoders = {}
x_obj = [c for c in X_train.columns if X_train[c].dtype == 'object']

for col in x_obj:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    encoders[col] = le

In [12]:
x_obj

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [13]:
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)

et = ExtraTreesClassifier(n_estimators=100)
et = et.fit(X_train, y_train)

In [14]:
joblib.dump(trn_mode, "./trn_mode.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(rf, "./random_forest.joblib", compress=True)
joblib.dump(et, "./extra_trees.joblib", compress=True)

['./extra_trees.joblib']

In [16]:
input_data = {
            "age": 37,
            "workclass": "Private",
            "fnlwgt": 34146,
            "education": "HS-grad",
            "education-num": 9,
            "marital-status": "Married-civ-spouse",
            "occupation": "Craft-repair",
            "relationship": "Husband",
            "race": "White",
            "sex": "Male",
            "capital-gain": 0,
            "capital-loss": 0,
            "hours-per-week": 68,
            "native-country": "United-States"
        }

In [20]:
def preprocessing(input_data):

        #JSON to pandas DataFrame
    input_data = pd.DataFrame(input_data, index=[0])
    input_data.fillna(trn_mode)

    for column in ["workclass", "education", "marital-status",
        "occupation", "relationship", "race", "sex", "native-country",]:
        lr = encoders[column]
        input_data[column] = lr.transform(input_data[column])

    return input_data

input_data = preprocessing(input_data)
input_data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,37,3,34146,11,9,2,2,0,4,1,0,0,68,37


In [21]:
rff = joblib.load("./random_forest.joblib")
rff.predict_proba(input_data)

array([[0.92, 0.08]])